# Copyright

Jelen iPython notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott "Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült a https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py linken elérhető kód alapján.

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot GPLv3 licensz védi. Újrafelhasználás esetén lehetőség szerint kérejük az alábbi szerzőt értesíteni.

2016 (c) Szaszák György (szaszak kukac tmit pont bme pont hu)


# Szöveggenerálás LSTM-mel

Karakterszekvenciákat tanulunk, ez alapján valamilyen seed-del indítva megtanítjuk a számítógépet szöveget írni.

In [50]:
# Importok
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

# Szövegkorpusz kiválasztása és letöltése
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('Karakterek száma a szövegben összesen:', len(text))

Karakterek száma a szövegben összesen: 600893


Karakter alapon dolgozunk, a karaktereket leképezzük egész azonosítókra.

In [51]:
chars = sorted(list(set(text)))
print('Előforduló karakterek száma:', len(chars))
# Szótárban a karakter-szám és az inverz leképezés
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print ("Karakterleképezések:", indices_char)

Előforduló karakterek száma: 57
Karakterleképezések: {0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '7', 18: '8', 19: '9', 20: ':', 21: ';', 22: '=', 23: '?', 24: '[', 25: ']', 26: '_', 27: 'a', 28: 'b', 29: 'c', 30: 'd', 31: 'e', 32: 'f', 33: 'g', 34: 'h', 35: 'i', 36: 'j', 37: 'k', 38: 'l', 39: 'm', 40: 'n', 41: 'o', 42: 'p', 43: 'q', 44: 'r', 45: 's', 46: 't', 47: 'u', 48: 'v', 49: 'w', 50: 'x', 51: 'y', 52: 'z', 53: 'ä', 54: 'æ', 55: 'é', 56: 'ë'}


A szöveget maxlen hosszú egységekbe tagoljuk. Ezt most sentence-nek nevezzük, de ez nem egyezik a valós mondatokkal. Tároljuk a következő karaktert, az lesz a target. Ezután step karakterrel odébb lépünk és megcsináljuk ugyanezt. step << maxlen, tehát nagy az átlapolás.

In [63]:
# A szöveget maxlen hosszú egységekbe tagoljuk. Ezt most sentence-nek nevezzük, de ez nem egyezik a valós mondatokkal.
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Tanítóminták száma:', len(sentences))
x_i = np.random.randint(0, len(sentences))
print('Egy random tanítóminta:', sentences[x_i])

Tanítóminták száma: 200285
Egy random tanítóminta: herein attainable, and also speaks of "t


Ahány karakterünk van, annyi dimenziós one hot lesz a bemenet. A betűk mellett minden más szövegben előforduló karaktert (írásjelek, esetleg számok, stb.) is használunk.

In [64]:
print('Adatok tenzorba rendezése...')
# X tanító, y target
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
# Feltöltjük one-hot kódolással
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print ("Tanító tenzor alakja:", X.shape)
print ("Teszt tenzor alakja:", y.shape)
print ("Az előző random minta 4. karakterének vektora: ", X[x_i,3,:], "A 4. karakter:", sentences[x_i][3])
print ("Az előző random minta targetje: ", y[x_i,:])

Adatok tenzorba rendezése...
Tanító tenzor alakja: (200285, 40, 57)
Teszt tenzor alakja: (200285, 57)
Az előző random minta 4. karakterének vektora:  [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False] A 4. karakter: e
Az előző random minta targetje:  [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False  True False
 False False False False False False False False False False False False
 False False False False False False False False False]


Összerakjuk a modellt

In [66]:
model = Sequential()
#Jön az LSTM réteg. Lehet több is, akkor meg kell tartanunk az összes kimenetet
#model.add(LSTM(128, input_shape=(maxlen, len(chars)),return_sequences=True))
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Tanítás előtt még írunk egy segédfüggvényt, amellyel mintát vehetünk egy eloszlásból. (Adott eloszlásból kiválasztjuk, hogy mégis melyik legyen az aktuális realizáció.) Így fogunk írni karakterről karakterre. A temperature változó hangolja, mennyire legyünk újító szelleműek (minél nagyobb, annál inkább felhúzzuk a kisebb valószínűségű karaktereket).

In [36]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds) # Az összes lehetőség egyre szummázódjon (lásd softmax képlet)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Tanítás, iterációnként szimuláljuk a működést. Elmentjük a modellt.

In [ ]:
for iteration in range(1, 60):
    print()
    print('=' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1) # a for ciklus vezérli az epochokat, itt futtatunk egy epochot
    
    modelfile="nietzsche_model_it_"+str(iteration)+".h5" # menthetjük a modellt, hogy később tudjuk futtatni
    model.save(modelfile)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]: # a mintavételhez kell majd (temperature)
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen] # seed random választva a szövegből
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400): # legenerálunk 400 karaktert egymás után
            x = np.zeros((1, maxlen, len(chars))) # ez megy a bemenetre
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1. # x-et feltöltjük karakter -> egész leképezéssel

            preds = model.predict(x, verbose=0)[0] # forward pass
            next_index = sample(preds, diversity) # kimeneten kapott eloszlásból mintát veszünk
            next_char = indices_char[next_index] # a mintát karakterre képezzük

            generated += next_char
            sentence = sentence[1:] + next_char # léptetünk egy karatert (első kiesik, utolsónak bejön a most generált)

            sys.stdout.write(next_char) # kiírjuk, amit generáltunk
            sys.stdout.flush()
        print()


Iteration 1
Epoch 1/1
 13568/200285 [=>............................] - ETA: 348s - loss: 1.6729

Korábban tanított modell betöltése és szöveggenerálás

In [55]:
from keras.models import load_model

startiteration = 1 # 5, 20, 40 vagy 60 lehet

model = load_model("nietzsche_model_it_"+str(startiteration)+".h5") # modell betöltése

for diversity in [0.2, 0.5, 1.0, 1.2]: # a mintavételhez kell majd (temperature)
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen] # seed random választva a szövegből
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400): # legenerálunk 400 karaktert egymás után
            x = np.zeros((1, maxlen, len(chars))) # ez megy a bemenetre
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1. # x-et feltöltjük karakter -> egész leképezéssel

            preds = model.predict(x, verbose=0)[0] # forward pass
            next_index = sample(preds, diversity) # kimeneten kapott eloszlásból mintát veszünk
            next_char = indices_char[next_index] # a mintát karakterre képezzük

            generated += next_char
            sentence = sentence[1:] + next_char # léptetünk egy karatert (első kiesik, utolsónak bejön a most generált)

            sys.stdout.write(next_char) # kiírjuk, amit generáltunk
            sys.stdout.flush()
        print()



----- diversity: 0.2
----- Generating with seed: "ay and believe--the jews performed the m"
ay and believe--the jews performed the morality of the ordeaing of the sensation of the most as the free spvenping the most an ancient extent to the sensation of the probably the subjection and actions and self-conscious and self-conscious of the soul of the problem of the sensation of the sense of the morality of the same as the sensation of the morality of the soul of the self-conception of the souls of the probably and as the sensiti

----- diversity: 0.5
----- Generating with seed: "ay and believe--the jews performed the m"
ay and believe--the jews performed the morality of the problem of the power as a platfory of an endorater of the drationing so in considerable the point, as it not to any honesty, and and bad acts and self-starious sense, in the uniformed to the freedom, and who has along hounds and everything of the ordeal think of the greater is to be what it is the probably to the vir